In [14]:
import pandas as pd
import math

a = [0] * 1000
a[40] = "Netherlands"
a[616] = "Poland"
a[36] = "Austria"
a[276] = "Germany"
a[56] = "Belgium"

class node:
    
    def __init__(self, arrive, depart, cash, counry_id):
        self.arrivial_time = arrive
        self.departure_time = depart
        self.cash = cash
        self.country_id = country_id


def radian(x):
    return (x * math.pi) / 180

def distance(f1, f2, q1, q2):
    f1 = radian(f1)
    f2 = radian(f2)
    q1 = radian(q1)
    q2 = radian(q2)
    temp1 = ((math.sin(f2 - f1) / 2)**2) + math.cos(f1) * math.cos(f2) * ((math.sin(q2 - q1) / 2)**2)
    return 2 * 6371 * math.asin(temp1**0.5)

def get_time(dist):
    return dist / 40

def id_to_sights(id):
    res = a[id]
    if (res == "Netherlands"):
        return pd.read_json('Netherlands')
    if (res == "Poland"):
        return pd.read_json('Poland')
    if (res == "Belgium"):
        return pd.read_json('Belgium')
    if (res == "Austria"):
        return pd.read_json('Austria')
    if (res == "Germany"):
        return pd.read_json('Germany')

In [18]:
start_time = pd.to_datetime("08-02-2020")
end_time = pd.to_datetime("09-02-2020")
id = 276
cash = 20000
node = (start_time, end_time, 20000, 40)
country = a[id]
id_to_sights(id)

,name,coordx,coordy,cost,time
0,Brandenburg Gate,52.516278,13.375515,0,900000
1,Reichstag,52.518620,13.376187,0,3600000
2,East Side Gallery,52.505408,13.436360,0,900000
3,Museum Island,52.516890,13.397764,1400,10800000
4,Unter den Linden,52.516848,13.387012,0,1200000
5,Kaiser Wilhelm Memorial Church,52.504821,13.332888,700,3600000
6,Postdam Platz,52.509652,13.373750,0,1800000
7,New Museum,52.520138,13.395452,1400,7200000
8,Checkpoint Charlie,52.507591,13.388181,0,900000
9,Berlin Flea Market,52.513591,13.331188,-1,2700000
